In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, names=rnames,  engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, names=mnames,  engine='python', encoding='ISO-8859-1')
data = pd.merge(pd.merge(ratings, users), movies)

# Exercises: 

**MovieLens database**

1- Filter  films that have received at least 250 ratings. **Use only these films in the rest of exercices**

In [2]:
def ex1():
    rated_films_count = data[['title', 'rating']].groupby('title').count()
    highly_rated_films = rated_films_count[rated_films_count['rating'] >= 250].sort_values(by='rating', ascending=False)
    
    return highly_rated_films

2- Obtain the mean ratings for each movie grouped by gender that have at least 250 ratings. 

In [3]:
def ex2():
    # Get the most rated films (num. ratings >= 250)
    highly_rated_films = ex1()
    
    # Filter the data, selecting first the most rated films and grouping the mean ratings
    # by title and gender
    filtered_data = data[data['title'].isin(highly_rated_films.index)][['title', 'gender', 'rating']].groupby(['title', 'gender']).mean()
    
    # Rearrange data in table so that the indexes are the titles and the columns are the mean ratings per gender
    mean_film_rating_per_gender = pd.pivot_table(filtered_data, values='rating', index=['title'], columns=['gender'])
    
    return mean_film_rating_per_gender

3- Show films more valued by women.

In [4]:
def ex3():
    mean_film_rating_per_gender = ex2()
    
    highest_rated_films_by_women = mean_film_rating_per_gender['F'].sort_values(ascending=False)
   
    return highest_rated_films_by_women

4- Now we wonder which movies are rated more differently between men and women. Which films have more different rating and are more highly valued by women? 

In [5]:
def ex4():
    mean_film_rating = ex2()
    mean_film_rating['difference'] = mean_film_rating['M'] - mean_film_rating['F']
    
    highly_rated_films_by_women = mean_film_rating.sort_values(by='difference')
    
    return highly_rated_films_by_women

5- And which films have more different rating and are more highly valued by men? 

In [6]:
def ex5():
    highly_rated_films_by_women = ex4()
    
    # We can simply reverse the DataFrame from the previous exercise!
    highly_rated_films_by_men = highly_rated_films_by_women[::-1]
    
    return highly_rated_films_by_men

6- What are the films that have generated the most discordant ratings, regardless of gender?

In [7]:
def ex6():
    highly_rated_films = ex1()
    
    films_ratings_std = data[data['title'].isin(highly_rated_films.index)][['title', 'rating']].groupby('title').std()
    most_discordant_films = films_ratings_std.sort_values(by='rating', ascending=False)
    
    return most_discordant_films

7- What is the highest rated movie in average?

In [8]:
def ex7():
    highly_rated_films = ex1()
    
    films_mean_ratings = data[data['title'].isin(highly_rated_films.index)][['title', 'rating']].groupby('title').mean()
    highest_rated_film = films_mean_ratings.sort_values(by='rating', ascending=False).head(1)
    
    return highest_rated_film

*** From here use all the ratings ***

8- Calculate the average rating of each user. 

In [9]:
def ex8():
    user_mean_score = data[['user_id', 'rating']].groupby('user_id').mean()
    
    return user_mean_score

9- Define a function called  <b>top_movies</b> that given a df, an user an a number n it returns what movies have the top n rank for this user.

def top_movies(data,user,n)


In [10]:
def top_movies(data_, usr, n_rows=10):
    user_top_ranked_movies = data_[data_['user_id'] == usr][['title', 'rating']].sort_values(by='rating', ascending=False).head(n_rows)
    
    return user_top_ranked_movies

def ex9():
    return top_movies(data, 1, 30)

** Data from CSV**

10- Read data from csv file: `ma-ba.csv`. Count the number of times `Barça` wins `Madrid` and compute the stadistics of % win, % lose and % draw.

In [11]:
def get_match_result(row):
    """
    Function to generate the winner of a football match.
    
    Generates the winner of a football match based on the number
    of goals each team has scored. Note that the teams can draw if
    none of them scores more goals than the other.
    
    Parameters
    ----------
        row: Series
            Row of a DataFrame to which the function is applied.
    
    Returns
    -------
    str
        String containing the winner of the match or 'Draw' if both teams have
        drawn.
    """
    if row.home_goals > row.away_goals:
        return row.home_team
    elif row.away_goals > row.home_goals:
        return row.away_team
    else:
        return 'Draw'

def ex10():
    # Read CSV file
    matches = pd.read_csv('ma-ba.csv', encoding='ISO-8859-1')
    
    num_matches = matches.shape[0]
    
    # Remove extra spaces from column names
    matches.columns = [col.strip() for col in matches.columns]
    
    # Process matches, removing extra spaces and creating an entry of the type 'Team A - Team B'
    # All 'R. Madrid' references are corrected
    matches['Partidos'] = [
        ' '.join(match.replace('Madrid FC', 'R. Madrid').replace('R.Madrid', 'R. Madrid').split())
        for match in matches['Partidos']
    ]
    
    matches[['home_team', 'away_team']] = matches['Partidos'].str.split(' - ', expand=True)
    matches[['home_goals', 'away_goals']] = matches['resultado'].str.split('-', expand=True).astype(int)
    matches['winner'] = matches.apply(get_match_result, axis=1)
    
    # Get count of how many times each team has won and how many draws there have been
    num_entries_results = matches['winner'].value_counts()
    
    barcelona_wins_loses = [num_entries_results['Barcelona'], num_entries_results['R. Madrid']]
    
    matches_statistics = pd.DataFrame(
        {
            'wins': barcelona_wins_loses,
            'loses': barcelona_wins_loses[::-1],
            'draws': [num_entries_results['Draw'] for _ in range(2)]
        }, index=['Barcelona', 'R. Madrid'])
    
    # We can easily get the % of wins, % of loses and % of draws for each team by dividing
    # the wins, loses and draws for each team by the number of matches they have played
    # Since these values form a 2x3 ndarray, the result will have the same shape
    matches_statistics[['perc_wins', 'perc_loses', 'perc_draws']] = matches_statistics / num_matches
    
    return matches_statistics

In [12]:
n = 11
for i in range(1,n):
    try:
        df = globals()["ex"+str(i)]()
        df.to_csv(str(i)+".csv",header=False)
    except Exception as e: 
        print(i,repr(e))
        open(str(i)+".csv","a").close()